In [3]:
from IPython.display import display, Markdown
import nbformat as nb

# Read the notebook file
with open('nav.ipynb', 'r') as file:
    notebook = nb.read(file, nb.NO_CONVERT)

# Extract and display Markdown cells
for cell in notebook.cells:
    if cell.cell_type == 'markdown':
        markdown_text = ''.join(cell.source)
        display(Markdown(markdown_text))

## Navigation
1. [Start here](hey.ipynb)
1. [Load data + exploratory analysis](eda.ipynb)
1. [Performance evalutation](eval_v1.ipynb)
1. [Gen: Faker Naive](faker_naive.ipynb)
1. [Gen: Faker Plus](faker_plus.ipynb)
1. [Gen: SDV Naive](sdv_v1.ipynb)
1. [Gen: SDV More Better](sdv_v2.ipynb)
1. [Gen: SDV TVAE]()
1. [Analysis: First impressions](eval_v2.ipynb)
1. [Analysis: Loan financial model](eval_v3.ipynb)
1. [Analysis: Predicting default risk](eval_v4.ipynb)
1. [Analysis: How hackable]()


# Load real loan data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats 
import matplotlib.pyplot as plt
import hvplot.pandas

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report, 
    roc_auc_score, roc_curve, auc,
    plot_confusion_matrix, plot_roc_curve
)
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

# Display all the things
pd.set_option('display.float', '{:.2f}'.format)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 200)

# "magic" command to make plots show up in the notebook
%matplotlib inline
sns.set_style('whitegrid')

In [2]:
dfa = pd.read_csv('INSERT FILE PATH', low_memory=False, compression='gzip')

## Clean and condition the raw dataset

In [ ]:
# Drop columns that are not needed for this exercise for various reasons
cols_to_drop = list(dfa.columns)

keepers = ['loan_amnt','term','int_rate','installment','sub_grade', 'dti_joint', 'annual_inc_joint',
            'emp_length','home_ownership','annual_inc','verification_status','issue_d',
            'loan_status','purpose','addr_state','dti','earliest_cr_line',
            'open_acc','pub_rec','revol_bal','revol_bal_joint','revol_util','total_acc','initial_list_status',
            'application_type','mort_acc','pub_rec_bankruptcies',
            'fico_range_low', 'fico_range_high', 'last_fico_range_low', 'last_fico_range_high',
            'settlement_amount', 'settlement_percentage', 'settlement_term']

for c in keepers:
    cols_to_drop.remove(c)

print(cols_to_drop)

In [ ]:
# Drop unneccesary columns
for c in cols_to_drop:
    if c in dfa.columns:
        dfa.drop(c, axis=1, inplace=True)

dfa.info()

### Drop small loans

In [5]:
# Drop loan_amnt under $1000
dfa = dfa[dfa['loan_amnt'] >= 1000]

### Transforming DTI and Annual Income for uniform analysis of all application types

In [ ]:
# View loan types value counts
dfa['application_type'].value_counts()

### Calculating unified DTI scores and annual incomes
Calculate Monthly Income:
- Convert annual income to monthly income by dividing by 12.
- For joint income, do the same.

Calculate Monthly Debt Payments:
- Convert the provided DTI values into monthly debt payments using the formula:
- Monthly Debt=DTI×Monthly Income

Aggregate Monthly Debt and Income:
- For joint applicants, total the monthly debt payments and total the monthly income from both individuals.

Calculate Combined DTI:
- Finally, compute the combined DTI using the total monthly debt and total monthly income:
- Combined DTI=Total Monthly Debt/Total Monthly Income×100

In [ ]:
"""
In summary, this code provides a way to calculate and unify the debt-to-income ratios 
for both individual and annual incomes for individuals and joint applicants while ensuring 
that missing values are handled appropriately. The final DataFrame retains only the 
unified values after dropping the original columns.
"""

# Create a copy of the original DataFrame 'dfa'
df = dfa.copy()

# Function to calculate unified DTI (Debt-to-Income ratio)
def calculate_unified_dti(row):
    # Calculate monthly incomes by dividing annual incomes by 12
    monthly_inc = row['annual_inc'] / 12 if row['annual_inc'] != 0 else 0
    monthly_inc_joint = row['annual_inc_joint'] / 12 if row['annual_inc_joint'] != 0 else 0
    
    # Calculate monthly debt payments based on DTI percentages
    monthly_debt = row['dti'] * monthly_inc / 100 if row['dti'] >= 0 else 0
    monthly_debt_joint = row['dti_joint'] * monthly_inc_joint / 100 if row['dti_joint'] >= 0 else 0

    # Total monthly debt is the sum of individual and joint monthly debt payments
    total_monthly_debt = monthly_debt + monthly_debt_joint

    # Total monthly income is the sum of individual and joint monthly incomes
    total_monthly_income = monthly_inc + monthly_inc_joint

    # Calculate unified DTI as a percentage of total monthly debt to total monthly income
    if total_monthly_income > 0:
        unified_dti = (total_monthly_debt / total_monthly_income) * 100
    else:
        unified_dti = np.nan  # Set to NaN if total income is zero

    # If joint DTI is missing or zero, use the individual's DTI
    if pd.isnull(row['dti_joint']) or row['dti_joint'] == 0:
        unified_dti = row['dti']  # Preserve the individual's DTI

    return unified_dti

# Function to calculate unified annual income
def calculate_unified_annual_inc(row):
    # If joint annual income is NaN or 0, return individual annual income
    if pd.isnull(row['annual_inc_joint']) or row['annual_inc_joint'] == 0:
        return row['annual_inc']  # Preserve the individual's annual income
    else:
        # Assume joint income includes individual income
        return row['annual_inc_joint']  # Return only the joint income

# Function to calculate unified revol_bal_joint
def calculate_unified_revol_bal(row):
    # If joint revol_bal is NaN or 0, return individual revol_bal
    if pd.isnull(row['revol_bal_joint']) or row['revol_bal_joint'] == 0:
        return row['revol_bal']  # Preserve the individual's revol_bal
    else:
        # Assume joint revol_bal includes individual revol_bal
        return row['revol_bal_joint']  # Return only the joint revol_bal

# Apply the functions to calculate unified DTI and unified annual income
df['unified_dti'] = df.apply(calculate_unified_dti, axis=1)
df['unified_annual_inc'] = df.apply(calculate_unified_annual_inc, axis=1)
df['unified_revol_bal'] = df.apply(calculate_unified_revol_bal, axis=1)

# Replace any remaining NaN values with 0 for specified columns
df.fillna(0, inplace=True)

# Write the unified values back to the original DataFrame 'dfa'
dfa['unified_dti'] = df['unified_dti']
dfa['unified_annual_inc'] = df['unified_annual_inc']
dfa['unified_revol_bal'] = df['unified_revol_bal']

# Drop the original DTI and income columns from the original DataFrame
dfa.drop(['dti', 'dti_joint', 'annual_inc', 'annual_inc_joint', 'revol_bal', 'revol_bal_joint'], axis=1, inplace=True)

# Optional: Print the updated DataFrame to verify changes
print(dfa[['unified_dti', 'unified_annual_inc', 'unified_revol_bal']])

### Condition and clean settlement data for all applicants (incl non-settling)

In [ ]:
# Condition and clean up settlement data
# If a loan did not settle, replace null values with 0
dfa[['settlement_amount', 'settlement_percentage', 'settlement_term']].value_counts()
# Create a new settlement flag boolean column with False for null OR 0 values in settlement_amount
dfa['settlement_flag'] = dfa['settlement_amount'].isnull() | (dfa['settlement_amount'] == 0)
print(dfa['settlement_flag'].value_counts())
# Fill null values in settlement_amount, settlement_term, and settlement_percentage with 0
dfa['settlement_amount'] = dfa['settlement_amount'].fillna(0)
dfa['settlement_term'] = dfa['settlement_term'].fillna(0)
dfa['settlement_percentage'] = dfa['settlement_percentage'].fillna(0)
print(dfa[['settlement_amount', 'settlement_percentage', 'settlement_term']].value_counts())

### Continue with standard data cleaning

In [ ]:
# Convert dates to dates in a sortable format
dfa['issue_d'] = pd.to_datetime(dfa['issue_d'],format='%b-%Y')
dfa['earliest_cr_line'] = pd.to_datetime(dfa['earliest_cr_line'],format='%b-%Y')

# Confirm the dtypes have been updated
dfa[['issue_d', 'earliest_cr_line']].info()

In [ ]:
# Average the fico ranges since these columns all give us the same info
dfa['fico_range_avg'] = dfa[['fico_range_low', 'fico_range_high']].mean(axis=1)
dfa.drop(['fico_range_low', 'fico_range_high'], axis=1, inplace=True)
dfa['fico_range_avg'].value_counts(sort=True, dropna=True)

In [ ]:
# Average the fico ranges since these columns all give us the same info
dfa['last_fico_range_avg'] = dfa[['last_fico_range_low', 'last_fico_range_high']].mean(axis=1)
dfa.drop(['last_fico_range_low', 'last_fico_range_high'], axis=1, inplace=True)
dfa['last_fico_range_avg'].value_counts(sort=True, dropna=False)

In [ ]:
# Replace null last_fico_range_avg with fico_range_avg
dfa['last_fico_range_avg'] = dfa['last_fico_range_avg'].fillna(dfa['fico_range_avg'])
dfa[['fico_range_avg', 'last_fico_range_avg']].value_counts(sort=True, dropna=True)

### Clean up loan status values

In [ ]:
# Reclassify loan status
dfa['loan_status'].value_counts()

In [ ]:
# Use regex to replace 'Does not meet the credit policy. Status:Charged Off' with 'Default'
import re
pattern = r'Does not meet the credit policy. Status:Charged Off'
dfa['loan_status'] = dfa['loan_status'].replace(pattern, 'Default', regex=True)
pattern = r'Does not meet the credit policy. Status:Fully Paid'
dfa['loan_status'] = dfa['loan_status'].replace(pattern, 'Fully Paid', regex=True)

# View all value counts including nulls
dfa['loan_status'].value_counts(dropna=True)

In [ ]:
# Re-categorize loans based on loan status
def categorize_loan_status(df):
    """Categorize loan statuses for analysis."""
    conditions = [
        df['loan_status'].isin(['Fully Paid','Does not meet the credit policy. Status:Fully Paid']),
        df['loan_status'].isin(['Current', 'In Grace Period']),
        df['loan_status'].isin(['Default', 'Charged Off', 'Does not meet the credit policy. Status:Charged Off']),
        df['loan_status'].isin(['Late (31-120 days)', 'Late (16-30 days)'])
    ]
    choices = ['Fully Paid', 'Current', 'Default', 'Late']
    
    df['loan_status'] = np.select(conditions, choices, default='Unknown')
    return df

categorize_loan_status(dfa)
# Drop all rows with a loan status of current or late
dfa = dfa[~dfa['loan_status'].isin(['Current', 'Late'])]
# Print loan_status and loan_category
dfa['loan_status'].value_counts()

In [ ]:
# Clean up the home ownership categories
dfa.loc[(dfa.home_ownership == 'ANY') | (dfa.home_ownership == 'NONE'), 'home_ownership'] = 'OTHER'  
dfa.home_ownership.value_counts()

In [ ]:
# Look at the value counts of all object columns
for c in dfa:
    if dfa[c].dtype == 'object':
        print(dfa[c].value_counts())

In [18]:
# Ensure that 'total_acc' and 'mort_acc' are numeric
dfa['total_acc'] = pd.to_numeric(dfa['total_acc'], errors='coerce')
dfa['mort_acc'] = pd.to_numeric(dfa['mort_acc'], errors='coerce')

# Calculate the average 'mort_acc' for each 'total_acc'
total_acc_avg = dfa.groupby(by='total_acc')['mort_acc'].mean()

def fill_mort_acc(total_acc, mort_acc):
    if np.isnan(mort_acc):
        return total_acc_avg.get(total_acc, np.nan)  # Use .get() to avoid KeyError
    else:
        return mort_acc

# Apply the function to fill missing 'mort_acc' values
dfa['mort_acc'] = dfa.apply(lambda x: fill_mort_acc(x['total_acc'], x['mort_acc']), axis=1)

In [19]:
# if the last_fico_range_avg is null, replace it with fico_range_avg
dfa['last_fico_range_avg'] = dfa['last_fico_range_avg'].fillna(dfa['fico_range_avg'])

# Drop those columns like hot potatoes
drops = ['last_fico_range_low', 'last_fico_range_high', 'fico_range_low', 'fico_range_high']
for d in drops:
    if d in dfa.columns:
        dfa.drop(d, axis=1, inplace=True)

In [ ]:
# See how many null or 0 values exist in the remaining columns
sns.displot(
    data=dfa.isnull().melt(value_name='missing'),
    y='variable',
    hue='missing',
    multiple='fill',
    height=8,
    aspect=1.1
)

# specifying a threshold value of 25% null values
plt.axvline(0.25, color='r')

In [21]:
# Replace null values in numeric columns with 0
if dfa.isnull().values.any() & dfa.dtypes.isin(['float64', 'int64']).any():
    dfa = dfa.fillna(0)

In [ ]:
# Drop the rows with over 75% null values
dfa = dfa.dropna(thresh=len(dfa)*0.75, axis=1)

# Look at the remaining null values
sns.displot(
    data=dfa.isnull().melt(value_name='missing'),
    y='variable',
    hue='missing',
    multiple='fill',
    height=8,
    aspect=1.1
)

In [ ]:
# Final look
dfa.info(verbose=True)

In [24]:
# Save cleaned data to the google drive
dfa.to_csv("INSERT FILE PATH", compression='gzip', index=False, header=True)

In [ ]:
# Save issue_d years in 2018 as a new dataframe
dfa_2018 = dfa.loc[dfa['issue_d'].dt.year == 2018]
dfa_2018.info(verbose=True)
dfa_2018.to_csv("INSERT FILE PATH", compression='gzip', index=False, header=True)

In [ ]:
# Remove the 2018 data from the original dataframe
dfa = dfa.loc[dfa['issue_d'].dt.year != 2018]
# Remove dates before 2013
dfa = dfa.loc[dfa['issue_d'].dt.year >= 2013]
dfa.info(verbose=True)

In [27]:
# Save our new source data for generating and training synthetic models
dfa.to_csv("INSERT FILE PATH", compression='gzip', index=False, header=True)

In [28]:
# Take a look at the memory usage after initial cleanup
dfa.memory_usage(index=False, deep=True).to_csv('INSERT FILE PATH')

In [ ]:
# Save the headers only to a csv
print(dfa.columns)
# Add values in the columns dataframe to a list and save to a CSV file
cols = pd.DataFrame(dfa.columns)
cols.to_csv("INSERT FILE PATH", index=False, header=True)